# Projet: Biological response prediction

In [2]:
from sklearn.ensemble import ExtraTreesClassifier
import matplotlib.pyplot as plt
import random
import pandas as pd
import numpy as np

## 1. Lecture du Data set

In [3]:
path_train = "../dataset/train.csv"
path_test = "../dataset/test.csv"
data_pd = pd.read_csv(path_train, sep=",") #The first column is the target
# We split the data for train set and a validation set
# Actually the validation set will act like a test set here
data = (data_pd.as_matrix()[:,:]).astype(float)
random.shuffle(data)
limit_train = int(0.8 * data.shape[0])
train_set = data[:limit_train]
valid_set = data[limit_train + 1:]

#X_complet, y_complet = data[:, 1:], data[:, 0]

X_train, y_train = train_set[:, 1:], train_set[:,0]
X_valid, y_valid = valid_set[:, 1:], valid_set[:, 0]

test_set = pd.read_csv(path_test, sep=",").as_matrix()
test_set_x = test_set[:, 1:]

# Exploration des données

In [28]:
data_pd.head()

(3751, 1777)

In [5]:
data_pd.describe()

,Activity,D1,D2,D3,D4,D5,D6,D7,D8,D9,...,D1767,D1768,D1769,D1770,D1771,D1772,D1773,D1774,D1775,D1776
count,3751.000000,3751.000000,3751.000000,3751.000000,3751.000000,3751.000000,3751.000000,3751.000000,3751.000000,3751.000000,...,3751.000000,3751.000000,3751.000000,3751.000000,3751.000000,3751.000000,3751.000000,3751.000000,3751.000000,3751.000000
mean,0.542255,0.076948,0.592436,0.068142,0.038990,0.212112,0.686653,0.274713,0.455133,0.749517,...,0.026926,0.014663,0.013863,0.021861,0.015196,0.016796,0.012263,0.011730,0.020261,0.011197
std,0.498278,0.079989,0.105860,0.078414,0.115885,0.102592,0.078702,0.090017,0.162731,0.071702,...,0.161889,0.120215,0.116938,0.146249,0.122348,0.128522,0.110074,0.107683,0.140911,0.105236
min,0.000000,0.000000,0.282128,0.000000,0.000000,0.002630,0.137873,0.006130,0.000000,0.275590,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.033300,0.517811,0.000000,0.000000,0.138118,0.625627,0.207374,0.378062,0.707339,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.000000,0.066700,0.585989,0.050000,0.000000,0.190926,0.674037,0.277845,0.499942,0.738961,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,1.000000,0.100000,0.668395,0.100000,0.000000,0.261726,0.740663,0.335816,0.569962,0.788177,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.000000,1.000000,0.964381,0.950000,1.000000,1.000000,0.994735,0.790831,0.989870,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


## 2. Mesure de l'importance des features _ Random forest 

In [9]:
# Build a forest and compute the feature importances
forest = ExtraTreesClassifier(n_estimators=250,
                              random_state=0)

forest.fit(X_train, y_train)
importances = forest.feature_importances_
std = np.std([tree.feature_importances_ for tree in forest.estimators_],
             axis=0)
indices = np.argsort(importances)[::-1]
indices_for_plot = indices[:50] # Afficher les 50 premiers seulement ! 

# Diagramme en barre permettant de connaitre l'importance des features
%matplotlib auto
df = pd.DataFrame( importances[indices_for_plot], index = indices_for_plot)
df.plot(kind='bar', legend = False, width = .8, figsize = (15,5))
plt.title("Features importance")
plt.show()

Using matplotlib backend: TkAgg


In [122]:
def split_dataset(nb_splits, data_X, data_Y): 
    nb_splits = nb_splits+2
    step_size = int(len(data)/nb_splits) 
    set_X = {}
    set_Y = {}
    deb = 0
    fin = step_size
    for i in range(nb_splits):
        set_X[i] = data_X[deb:fin]
        set_Y[i] = data_Y[deb:fin]
        deb = fin 
        fin = fin + step_size
    return set_X, set_Y

nb_splits = 10
set_X, set_Y = split_dataset(nb_splits, X_train, y_train)

ranking_lists = []
for i in range(nb_splits):
    # Build a forest and compute the feature importances
    forest = ExtraTreesClassifier(n_estimators=250, random_state=None)
    forest.fit(set_X[i], set_Y[i])
    std = np.std([tree.feature_importances_ for tree in forest.estimators_],axis=0)
    ranking_lists.append(np.argsort(importances)[::-1])
    
#Pour le nom de la mesure: 
# "spearman_correlation"
# "Jaccard_RankBased_index"
# "kuncheva_index: "
print stability_measure(ranking_lists, "spearman_correlation", 100)

spearman_correlation
1.0


# Mesures de stabilité 

In [115]:
# Mesure de stabilité - Comparaison deux à deux moyennée
def stability_measure(feature_selection_matrix, measure_name, rank):
    L = len(feature_selection_matrix)
    norm = 2.0/(L*(L-1))
    
    sum_i = 0
    for i in range(0,L-1):
        sum_j = 0
        for j in range(i+1,L):
            if measure_name == "spearman_correlation":
                sum_j += spearman_correlation(feature_selection_matrix[i], feature_selection_matrix[j])
            elif measure_name == "Jaccard_RankBased_index":
                sum_j += Jaccard_RankBased_index(feature_selection_matrix[i], feature_selection_matrix[j], rank)
            else: 
                sum_j +=  kuncheva_index(feature_selection_matrix[i], feature_selection_matrix[j], rank)
        sum_i += sum_j
    print measure_name                  
    return sum_i*norm
    

## 1. Fully Ranking metrics

In [120]:
#Spearman correlation
from scipy.stats import spearmanr
def spearman_correlation(selection_i, selection_j):
    N = len(selection_i)
    sum_spearman = 0
    for i in range(N):
        sum_spearman += 6*((i-selection_j[np.where( selection_j == selection_i[i])][0] ))^2 / (N*(N^2 - 1))
    return 1 - sum_spearman

## 2. Partial Ranking Metrics

In [117]:
#Jaccard rank-based index
def Jaccard_RankBased_index(selection_i, selection_j,rank):
    sum_sim = 0
    selection_j = selection_j[:rank]
    for indice in range(rank):
        if selection_i[indice] in selection_j:
            sum_sim += 1.0/(2*rank-1)
    return sum_sim

#Kuncheva index
def kuncheva_index(selection_i, selection_j,rank):
    N = len(selection_i)
    sum_sim = 0
    for indice in range(rank):
        if selection_i[indice] in selection_j[:rank]:
            sum_sim += (1.0-(rank*rank)/N)/(rank-(rank*rank)/N)
    return sum_sim

# Variance Treshold

In [19]:
from sklearn.feature_selection import VarianceThreshold
print X_train.shape
selector = VarianceThreshold()
selector.fit_transform(X_train)
print X_train.shape

(3000, 1776)
(3000, 1776)
